# "Extract DataFrame from Compressed Data into Python Datatable"
> "Extract tables from compressed data with ease"

- toc: true
- branch: master
- badges: true
- hide_binder_badge: True
- hide_colab_badge: True
- comments: true
- author: Samuel Oranyeli
- categories: [python, compressed data, zip, archived data, pydatatable, rdatatable]
- image: images/some_folder/your_image.png
- hide: false
- search_exclude: true
- metadata_key1: "pydatatable"
- metadata_key2: "compressed data"

## **Compressed Data Extraction in Python Datatable**

In a previous [blog post](https://samukweku.github.io/data-wrangling-blog/python/pandas/compressed%20data/2020/07/21/Extract-DataFrame-from-Compressed-Data-into-Pandas.html), I discussed the various ways to extract data  from compressed files into [Pandas](https://pandas.pydata.org/pandas-docs/stable/index.html). This blog post explores those options, this time with [Datatable](https://datatable.readthedocs.io/en/latest/?badge=latest). This package is a reimplementation in Python of [Rdatatable](https://rdatatable.gitlab.io/data.table/), aiming to replicate its speed and concise syntax. Have a look at my other [blog post](https://samukweku.github.io/data-wrangling-blog/python/datatable/2020/06/25/Selecting-and-Grouping-Data-with-Python-Datatable.html) that explores the basics of [Datatable](https://datatable.readthedocs.io/en/latest/?badge=latest).
Most of the examples will focus on the ``zip`` format, as it is widely used and the methods presented here can be adapted for other compression formats. 

### __Read Zip Folder that has only one File__

![zip-single-file.png](Images/zip_single_file.png)

We simply pass the zipfile : 

In [1]:
from datatable import dt, fread, iread

df = fread("Data_files/iris.zip")

df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5,3.6,1.4,0.2,setosa
5,5.4,3.9,1.7,0.4,setosa
6,4.6,3.4,1.4,0.3,setosa
7,5,3.4,1.5,0.2,setosa
8,4.4,2.9,1.4,0.2,setosa


**Notes: **
- [fread](https://datatable.readthedocs.io/en/latest/api/dt/fread.html#) and [iread](https://datatable.readthedocs.io/en/latest/api/dt/iread.html#) are file reading features of [Datatable](https://datatable.readthedocs.io/en/latest/?badge=latest), similar to [Pandas](https://pandas.pydata.org/pandas-docs/stable/index.html) [read_csv](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html)

### __Read a File from Multiple Files in Zip Folder__

![zip-single-file-from-multiple-files.png](Images/zip_single_from_multiple_files.png)

In this scenario, We want just the ``setosa.csv`` file. [Datatable](https://datatable.readthedocs.io/en/latest/?badge=latest) can read this with ease - simply pass the filepath to [fread](https://datatable.readthedocs.io/en/latest/api/dt/fread.html#) -> ``archive_name.zip/path_to_file``:

In [2]:
df = fread("Data_files/iris_zipped.zip/iris_zipped/setosa.csv")

df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5,3.6,1.4,0.2,setosa
5,5.4,3.9,1.7,0.4,setosa
6,4.6,3.4,1.4,0.3,setosa
7,5,3.4,1.5,0.2,setosa
8,4.4,2.9,1.4,0.2,setosa


This is easier than working with [Pandas](https://pandas.pydata.org/pandas-docs/stable/index.html), where we had to import the [zipfile](https://docs.python.org/3/library/zipfile.html) to do this.

### __Read a Specific File from Multiple Files in Zip Folder based on its Extension__

![zip-specific-file-ext.png](Images/zip_specific_file_extension.png)

Here, the zip folder contains various file types (``xlsx``, ``csv``, ``json``, ...), and we want only the ``.csv`` file. Let's also assume that we do not know beforehand what the filename is; we do however know that there is just one ``.csv`` file in the zip folder. There are two options : 

Option 1 : We can pick out the specific file with the [iread](https://datatable.readthedocs.io/en/latest/api/dt/iread.html#) function. [iread](https://datatable.readthedocs.io/en/latest/api/dt/iread.html#) returns a lazy iterator of all the data frames in the zip folder; as such we can filter for the specific file we are interested in :

In [3]:
for file in iread("Data_files/zipped.zip"):
    # every file has a source attribute
    # that describes the file,
    # including the file type and name
    if file.source.endswith(".csv"):
        df = file
        
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5,3.6,1.4,0.2,setosa
5,5.4,3.9,1.7,0.4,setosa
6,4.6,3.4,1.4,0.3,setosa
7,5,3.4,1.5,0.2,setosa
8,4.4,2.9,1.4,0.2,setosa


**Note:**
- The ``.json`` format is not supported by [Datatable](https://datatable.readthedocs.io/en/latest/?badge=latest).

Option 2 : Use the ``cmd`` argument. This passes the processing to the command line. The command line is a powerful tool that can help with a lot of data processing. No need for the [iread](https://datatable.readthedocs.io/en/latest/api/dt/iread.html#) function, just let the command line do the work.

In [4]:
df = fread(cmd = """unzip -p Data_files/zipped.zip '*.csv' """)

df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5,3.6,1.4,0.2,setosa
5,5.4,3.9,1.7,0.4,setosa
6,4.6,3.4,1.4,0.3,setosa
7,5,3.4,1.5,0.2,setosa
8,4.4,2.9,1.4,0.2,setosa


**Note :**
- In the code above, we used the [unzip](https://linux.die.net/man/1/unzip) tool to filter for the ``csv`` file and extract the data. 

### __Read Multiple Files with specific extension in Zip Folder into a Single Dataframe__

![zip-multiple-files-concat.png](Images/zip_multiple_files_concat.png)

In this scenario, we want to read in only the ``csv`` files and combine them into one dataframe - We will explore the [iread](https://datatable.readthedocs.io/en/latest/api/dt/iread.html#) option and the command line option:

Option 1 - [iread](https://datatable.readthedocs.io/en/latest/api/dt/iread.html#) :

In [5]:
csv_files = [file for file in iread("Data_files/zip_various.zip")
             if file.source.endswith(".csv")]

# combine the dataframes into one with rbind 
# rbind is similar to pd.concat(*frames, axis=0) :
df = dt.rbind(csv_files)

df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5,3.6,1.4,0.2,setosa
5,5.4,3.9,1.7,0.4,setosa
6,4.6,3.4,1.4,0.3,setosa
7,5,3.4,1.5,0.2,setosa
8,4.4,2.9,1.4,0.2,setosa


Option 2 - Command Line :

In [6]:
df = fread(cmd = """
                 unzip -p Data_files/zip_various.zip '*.csv' | 
                 awk 'BEGIN {FS = ","}; {if ($1!="sepal_length" || NR==1) {print}}' 
                """)

df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5,3.6,1.4,0.2,setosa
5,5.4,3.9,1.7,0.4,setosa
6,4.6,3.4,1.4,0.3,setosa
7,5,3.4,1.5,0.2,setosa
8,4.4,2.9,1.4,0.2,setosa


**Note:**
- The [AWK](https://www.gnu.org/software/gawk/manual/html_node/Getting-Started.html#Getting-Started) language is used here to split the columns on the `,` delimiter, and to get the header for the rows.

### __Read all the Files in Zip Folder into a Single Dataframe__

![zip-single-file-from-multiple-files.png](Images/zip_single_from_multiple_files.png)

Option 1 - [iread](https://datatable.readthedocs.io/en/latest/api/dt/iread.html#) :

In [7]:
#get a sequence of the files and combine with rbind
df = dt.rbind(tuple(iread("Data_files/iris_zipped.zip")))

df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5,3.6,1.4,0.2,setosa
5,5.4,3.9,1.7,0.4,setosa
6,4.6,3.4,1.4,0.3,setosa
7,5,3.4,1.5,0.2,setosa
8,4.4,2.9,1.4,0.2,setosa


Option 2 - Command Line :

In [8]:
df = fread(cmd = """
                 unzip -p Data_files/iris_zipped.zip | 
                 awk 'BEGIN {FS = ","}; {if ($1!="sepal_length" || NR==1) {print}}' 
                 """)

df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5,3.6,1.4,0.2,setosa
5,5.4,3.9,1.7,0.4,setosa
6,4.6,3.4,1.4,0.3,setosa
7,5,3.4,1.5,0.2,setosa
8,4.4,2.9,1.4,0.2,setosa


### __What about other Formats?__

The major compression archive formats supported by ``fread`` are : ``.tar``, ``.gz``, ``.zip``, ``.gz2``, and ``.tgz``. So the above steps will work for these formats. But there are other compresssion formats, including ``.7z``. How do we read that? The command line!

- Extract archive with a single file :

![7z-single-file.png](Images/7z_single.png)

In [9]:
# kindly ensure 7z is already installed on your system
df = fread(cmd = "7z e -so Data_files/iris_single_7z.7z")

df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5,3.6,1.4,0.2,setosa
5,5.4,3.9,1.7,0.4,setosa
6,4.6,3.4,1.4,0.3,setosa
7,5,3.4,1.5,0.2,setosa
8,4.4,2.9,1.4,0.2,setosa


- Extract specific file :

![7z_multiple.png](Images/7z_multiple.png)

In [10]:
df = fread(cmd = """ 7z e -so Data_files/iris_7z.7z "virginica.csv" -r """)

df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪
0,6.3,3.3,6,2.5,virginica
1,5.8,2.7,5.1,1.9,virginica
2,7.1,3,5.9,2.1,virginica
3,6.3,2.9,5.6,1.8,virginica
4,6.5,3,5.8,2.2,virginica
5,7.6,3,6.6,2.1,virginica
6,4.9,2.5,4.5,1.7,virginica
7,7.3,2.9,6.3,1.8,virginica
8,6.7,2.5,5.8,1.8,virginica


### __Summary__

[Datatable](https://datatable.readthedocs.io/en/latest/?badge=latest) offers convenient features in reading data from compressed file formats, including the ability to read in data via the command line. Compared to [Pandas](https://pandas.pydata.org/pandas-docs/stable/index.html), [Datatable](https://datatable.readthedocs.io/en/latest/?badge=latest) offers more flexibility and power when dealing with compressed file formats.